# Problema básico de la mezcla


La Facultad de Ciencias Agrarias de la Universidad Nacional necesita producir un suplemento alimenticio balanceado para el ganado de su estación experimental. El suplemento debe contener nutrientes esenciales como calcio, proteínas y fibra, para garantizar el adecuado crecimiento y desarrollo del ganado.

Se tienen tres ingredientes disponibles para la mezcla del suplemento: piedra caliza, maíz y harina de soya. Cada ingrediente tiene diferentes proporciones de calcio, proteínas y fibra, así como costos por kilogramo distintos. El objetivo es determinar las cantidades óptimas de cada ingrediente que deben mezclarse para lograr el suplemento más económico, cumpliendo con los requisitos nutricionales del ganado.

## Datos

| Ingrediente       | Calcio (kg/kg) | Proteínas (kg/kg) | Fibra (kg/kg) | Costo (cents/kg) |
|-------------------|---------------|-------------------|--------------|------------------|
| Piedra Caliza     | 0.38          | 0.0               | 0.0          | 10.0             |
| Maíz              | 0.001         | 0.09              | 0.02         | 30.5             |
| Harina de Soya    | 0.002         | 0.50              | 0.08         | 90.0             |

La mezcla, por requerimientos, debe seguir las siguiente composición:

- Calcio mínimo requerido: por lo menos 0.8% pero no más de 1.2%
- Proteínas mínimas requeridas: por lo menos 22%
- Fibra mínima requerida: A lo máximo 5%

El problema consiste en encontrar la composición que satisfaga las restricciones mientras minimiza el costo.


## Modelo Matemático


### Variables de decisión
$x_1, x_2, x_3$: Cantidad de cada componente a mezclar (enteros positivos)

$x_1 = $ Piedra caliza

$x_2 = $ Maíz

$x_3 = $ Harina de soya


### Restricciones

**Restricciones de calcio**:

$ 0.38  x_1 + 0.001  x_2 + 0.002  x_3 \geq 0.008 $

$ 0.38  x_1 + 0.001  x_2 + 0.002  x_3 \leq 0.012 $

**Restricción de Proteínas**:

$ 0.0  x_1 + 0.09  x_2 + 0.50  x_3 \geq 0.22$

**Restricción de Fibra**:

$0.0  x_1 + 0.02  x_2 + 0.08  x_3 \leq 0.05$

**Conservacion**

En este caso lo modelaremos para solo realizar 1kg de la mezcla

$x_1 + x_2 + x_3 = 1$

**No Negatividad**

$x_1,x_2,x_3 \geq 0$

### Función objetivo

$Min$
$ 10x_1 + 30.5x_2 + 90x_3$

## Solución

In [ ]:
import cvxpy as cvx

x_1 = cvx.Variable()
x_2 = cvx.Variable()
x_3 = cvx.Variable()

costo_total = cvx.Minimize(10 * x_1 + 30.5 * x_2 + 90 * x_3)

restricciones = [
    0.38 * x_1 + 0.001 * x_2 + 0.002 * x_3 <= 0.012,
    0.38 * x_1 + 0.001 * x_2 + 0.002 * x_3 >= 0.008,
    0.0  * x_1 + 0.09  * x_2 + 0.50  * x_3 >= 0.22,
    0.0  * x_1 + 0.02  * x_2 + 0.08  * x_3 <= 0.05,
    x_1 + x_2 + x_3 == 1,
    x_1 >=0,
    x_2 >=0,
    x_3 >=0
]

# Problema de optimización
problema = cvx.Problem(costo_total, restricciones)

# Resolver el problema
resultado = problema.solve()

# Chequeamos el status del problema
print("El estado de la solución es: " + problema.status + "\n")

# Chequeamos el status del problema
print("Valor de la función objetivo ", problema.value)
print("Piedra caliza:", x_1.value)
print("Maiz: ", x_2.value)
print("Harina de soya: ", x_3.value)

El estado de la solución es: optimal

Valor de la función objetivo  49.15629019605402
Piedra caliza: 0.02817082513628019
Maiz:  0.6485721645883664
Harina de soya:  0.32325701027495607


## Otra forma de resolverlo

In [ ]:
# importar librerias a utilizar
import cvxpy as cvx
import numpy as np


# Datos
tabla_ingredientes = [
    {
        "Ingrediente": "Piedra Caliza",
        "Calcio (kg/kg)": 0.38,
        "Proteínas (kg/kg)": 0.0,
        "Fibra (kg/kg)": 0.0,
        "Costo (cents/kg)": 10.0,
    },
    {
        "Ingrediente": "Maíz",
        "Calcio (kg/kg)": 0.001,
        "Proteínas (kg/kg)": 0.09,
        "Fibra (kg/kg)": 0.02,
        "Costo (cents/kg)": 30.5,
    },
    {
        "Ingrediente": "Harina de Soya",
        "Calcio (kg/kg)": 0.002,
        "Proteínas (kg/kg)": 0.50,
        "Fibra (kg/kg)": 0.08,
        "Costo (cents/kg)": 90.0,
    },
]

# Extraer solo los valores numéricos en una lista de la lista de ingredientes
valores_numericos = [[valor for clave, valor in ingrediente.items()\
                    if isinstance(valor, (int, float))]\
                    for ingrediente in tabla_ingredientes]

# Convertir la lista en un array de NumPy (Esta contiene los costos)
# Pero ya se los vamos a extraer
array_coef_tec = np.array(valores_numericos)
costos = array_coef_tec[:, 3]

# Ahora dejo solo el array de coeficientes técnicos
array_coef_tec = array_coef_tec[:, :-1]

# Requerimientos nutricionales mínimos
calcio_minimo = 0.008  # 0.8%
calcio_maximo = 0.012  # 1.2%
prote_maximo  = 0.22
fibra_minimo  = 0.005

# Variables de decisión
cantidades_ingredientes = cvx.Variable(( 3,1 ), nonneg = True)

# Función objetivo (minimizar el costo total)
costo_total = cvx.Minimize(costos @ cantidades_ingredientes)

# Se crea la lista donde se almacenarán las restricciones
restricciones = []

# Restriccion de conservacion
restricciones.append( cvx.sum(cantidades_ingredientes) == 1)

# Restricciones de calcio
suma_var_calcio = cvx.sum( array_coef_tec[:, 0] @ cantidades_ingredientes )
suma_var_prote  = cvx.sum( array_coef_tec[:, 1] @ cantidades_ingredientes )
suma_var_fibra  = cvx.sum( array_coef_tec[:, 2] @ cantidades_ingredientes )

# Añado las restricciones a las listas
restricciones.append( suma_var_calcio >= calcio_minimo)
restricciones.append( suma_var_calcio <= calcio_maximo)
restricciones.append( suma_var_prote >= prote_maximo )
restricciones.append( suma_var_fibra >= fibra_minimo )

# Problema de optimización
problema = cvx.Problem(costo_total, restricciones)

# Resolver el problema
resultado = problema.solve()

# Chequeamos el status del problema
print("El estado de la solución es: ", problema.status, "\n")

# Imprimimos el valor óptimo de la función objetivo
print("Valor de la función objetivo ", problema.value,"\n" )

# Imprimimos los valores óptimos de las variables de decisión
print("Variable Piedra Caliza: ", cantidades_ingredientes[0].value)
print("Variable Maiz: ", cantidades_ingredientes[1].value)
print("Variable Harina de soya: ", cantidades_ingredientes[2].value)

El estado de la solución es:  optimal 

Valor de la función objetivo  49.15629019520371 

Variable Piedra Caliza:  [0.02817083]
Variable Maiz:  [0.64857216]
Variable Harina de soya:  [0.32325701]
